In [1]:
!pip install -q -U google-generativeai

In [2]:
import google.generativeai as genai

## SET API KEY

In [3]:
# Used to securely store your API key
from google.colab import userdata

GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')

genai.configure(api_key=GOOGLE_API_KEY)

## LIST OF MODELS

In [4]:
for m in genai.list_models():
  if 'generateContent' in m.supported_generation_methods:
    print(m.name)


models/gemini-pro
models/gemini-pro-vision


In [5]:
# Model Configuration
MODEL_CONFIG = {
  "temperature": 0.2,
  "top_p": 1,
  "top_k": 32,
  "max_output_tokens": 4096,
}

## Safety Settings of Model
safety_settings = [
  {
    "category": "HARM_CATEGORY_HARASSMENT",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE"
  },
  {
    "category": "HARM_CATEGORY_HATE_SPEECH",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE"
  },
  {
    "category": "HARM_CATEGORY_SEXUALLY_EXPLICIT",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE"
  },
  {
    "category": "HARM_CATEGORY_DANGEROUS_CONTENT",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE"
  }
]

## LOAD GEMINI MODEL WITH MODEL CONFIGURATIONS

In [6]:
model = genai.GenerativeModel(model_name = "gemini-pro-vision",
                              generation_config = MODEL_CONFIG,
                              safety_settings = safety_settings)

## DEFINE IMAGE FORMAT TO INPUT IN GEMINI

In [7]:
from pathlib import Path

def image_format(image_path):
    img = Path(image_path)

    if not img.exists():
        raise FileNotFoundError(f"Could not find image: {img}")

    image_parts = [
        {
            "mime_type": "image/png", ## Mime type are PNG - image/png. JPEG - image/jpeg. WEBP - image/webp
            "data": img.read_bytes()
        }
    ]
    return image_parts


## GEMINI MODEL OUTPUT

In [8]:
def gemini_output(image_path, system_prompt, user_prompt):

    image_info = image_format(image_path)
    input_prompt= [system_prompt, image_info[0], user_prompt]
    response = model.generate_content(input_prompt)
    return response.text

### EXTRACTING PART OF THE INFORMATION FROM INVOICE

In [11]:
system_prompt = """
               You are a specialist in comprehending receipts.
               Input images in the form of receipts will be provided to you,
               and your task is to respond to questions based on the content of the input image.
               """

image_path = "/content/handwritten.png"

user_prompt = "What is the balance amount in the image?"

gemini_output(image_path, system_prompt, user_prompt)

' The balance amount is 370.00.'

## EXTRACTING WHOLE DATA IN JSON FROM INVOICE

In [12]:
system_prompt = """
               You are a specialist in comprehending receipts.
               Input images in the form of receipts will be provided to you,
               and your task is to respond to questions based on the content of the input image.
               """
#system_prompt = "Convert Invoice data into json format with appropriate json tags as required for the data in image "
image_path = "/content/handwritten.png"
user_prompt = "Convert Invoice data into json format with appropriate json tags as required for the data in image "

In [13]:

output = gemini_output(image_path, system_prompt, user_prompt)

In [14]:
from IPython.display import Markdown
Markdown(output)

 ```json
{
  "invoice_number": "8841",
  "invoice_date": "19/12/05",
  "customer_name": "M. Kennedy",
  "items": [
    {
      "description": "Brado 42 Black",
      "quantity": 1,
      "price": 500
    },
    {
      "description": "1 1/2\" smaller band",
      "quantity": 1,
      "price": 0
    },
    {
      "description": "3/4\" bigger neck",
      "quantity": 1,
      "price": 0
    },
    {
      "description": "Lemons collar",
      "quantity": 1,
      "price": 0
    },
    {
      "description": "No epaulettes",
      "quantity": 1,
      "price": 0
    },
    {
      "description": "Action Back",
      "quantity": 1,
      "price": 0
    },
    {
      "description": "2 inside pockets with zips",
      "quantity": 1,
      "price": 20
    }
  ],
  "total": 570,
  "deposit": 200,
  "balance": 370
}
```